### Prudhviraj Tirumanisetti 
#### EE 258 ID: 011489881

# Assignment 2: Evaluation and Decision Trees
In this assignment, we'll look at different ways of splitting data for training and testing, assessing the predictions made by data mining tools, and then explore classification using decision trees. Like Assignment 1, you will be expected to complete the assignment by answering the questions in Markdown cells and supporting your answer with corresponding code cells. 

Please submit your solution as two iPython notebooks (for Part 1 and Part 2) . The due date for this assignment is officially 9/27/16 at 11:59pm, but this may be extended based on the feedback I receive on the Canvas survey about projects and course pace, so keep an eye out for that survey.

# Part 1: Cross-validation and Evaluation Metrics (40 points)
Let's start by checking out the dataset evaluation tools availble in scikit-learn. The [cross-validation](http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation) documentation will be helpful to review before completing this part of the assignment. We'll work with two datasets, one  generated and one real dataset, and then look at different ways of splitting that data into train and test sets, as well as computing evaluation metrics on each of the datasets. 

In [9]:
## Preliminaries

#Show plots in the notebook
%matplotlib inline

from sklearn import datasets, preprocessing, cross_validation, metrics, utils, dummy, tree
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib2

In [10]:
# Let's get our first dataset, the Iris data that's the default choise for demos
iris = datasets.load_iris()

# Let's also generate some data. See http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html#sklearn.datasets.make_classification if you want to learn how this data is generated
(gendata_d, gendata_t) = datasets.make_classification(n_samples=500, random_state=20160121)
gendata_data = pd.DataFrame(gendata_d)
gendata_targets = pd.DataFrame(gendata_t)

In [11]:
# We'll start with the simplest holdout procedure: a single train-test split
[iris_train_data, iris_test_data,  iris_train_labels, iris_test_labels] = cross_validation.train_test_split(pd.DataFrame(iris.data), iris.target, test_size=0.25, random_state=20160121)

# Some notes on how this works:
## There are four outputs. The first is the dataframe containing the training data -- just the attributes. 
## The second output is the dataframe containing the testing data (attributes). The third and fourth outputs 
## are the training labels (used to train your classifier) and the testing labels (used to evaluate the classifier)

## The first two arguments to the split are all the data (attributes only) and all of the labels
## The test size is the fraction of data included in the test set, 25% in this case
## Since we're randomly sampling data, there's a chance every student would get different results.
##  To avoid this, we pass a random_state (20160121) to make sure the "random" output is the same for everyone

#Let's see how big the train and test sets are:
print iris_train_data.shape, iris_test_data.shape, iris_train_labels.shape, iris_test_labels.shape


(112, 4) (38, 4) (112,) (38,)


## Question 1: Creating a simple train-test split (5 points)
Use the `train_test_split` function (see the [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.train_test_split.html#sklearn.cross_validation.train_test_split)) to generate training and testing sets for the Iris and generated data.
1.  Generate splits with 10%, 33%, and 50% test data for both datasets (using `random_state=20160121` as a parameter) 
2.  Compute the percentage of the training labels and testing labels that belong to each class for each split. Does the distribution of labels appear to be well-matched between the train and test set?

3.  Compute the descriptive statistics for each feature in the train and test set for the 10% split. Which feature has the biggest difference in mean value across datasets.

<h1> Question 1 Answers </h1>
<p>1. see code below </p>
<p>2. For the IRIS dataset the distribution of the labels appears to be well-matched for the 10% and 50% split, but unbalanced for the 33% split
<br>
For the RAND Dataset the distribution of the labels does not appear to be as closely matched as the IRIS dataset. While the values are relatively close to each other, the test labels distribution does not match the train set label distribution</p>
<p>3. 
For the IRIS data set the biggest difference in mean value is the 3rd feature petal width: 1.2074 -> 1.12 <br>
For the RAND dataset the biggest difference in mean value is in attribue 8 : 0.018226 -> -0.30529

<br>See cell below for better formatting</p>

In [12]:
# Iris CV
[iris_train_data_10, iris_test_data_10,  iris_train_labels_10, iris_test_labels_10] = cross_validation.train_test_split(pd.DataFrame(iris.data), pd.DataFrame(iris.target), test_size=0.10, random_state=20160121)
[iris_train_data_33, iris_test_data_33,  iris_train_labels_33, iris_test_labels_33] = cross_validation.train_test_split(pd.DataFrame(iris.data), pd.DataFrame(iris.target), test_size=0.33, random_state=20160121)
[iris_train_data_50, iris_test_data_50,  iris_train_labels_50, iris_test_labels_50] = cross_validation.train_test_split(pd.DataFrame(iris.data), pd.DataFrame(iris.target), test_size=0.50, random_state=20160121)

# Gendata CV
[gendata_train_data_10, gendata_test_data_10,  gendata_train_labels_10, gendata_test_labels_10] = cross_validation.train_test_split(gendata_data, gendata_targets, test_size=0.10, random_state=20160121)
[gendata_train_data_33, gendata_test_data_33,  gendata_train_labels_33, gendata_test_labels_33] = cross_validation.train_test_split(gendata_data, gendata_targets, test_size=0.33, random_state=20160121)
[gendata_train_data_50, gendata_test_data_50,  gendata_train_labels_50, gendata_test_labels_50] = cross_validation.train_test_split(gendata_data, gendata_targets, test_size=0.50, random_state=20160121)

In [13]:
all_datasets = {'gendata_50': { 'train': gendata_train_labels_50, 'test': gendata_test_labels_50},
                'gendata_33': { 'train': gendata_train_labels_33, 'test': gendata_test_labels_33},
                'gendata_10': { 'train': gendata_train_labels_10, 'test': gendata_test_labels_10},
                'iris_50': { 'train': iris_train_labels_50, 'test': iris_test_labels_50},
                'iris_33': { 'train': iris_train_labels_33, 'test': iris_test_labels_33},
                'iris_10': { 'train': iris_train_labels_10, 'test': iris_test_labels_10}
               }
for dataset in all_datasets.keys():
    print dataset
    print all_datasets[dataset]['train'][0].value_counts(sort=False)
    print all_datasets[dataset]['test'][0].value_counts(sort=False)
    print "\n"


gendata_50
0    129
1    121
Name: 0, dtype: int64
0    120
1    130
Name: 0, dtype: int64


iris_33
0    32
1    34
2    34
Name: 0, dtype: int64
0    18
1    16
2    16
Name: 0, dtype: int64


gendata_10
0    226
1    224
Name: 0, dtype: int64
0    23
1    27
Name: 0, dtype: int64


iris_10
0    45
1    45
2    45
Name: 0, dtype: int64
0    5
1    5
2    5
Name: 0, dtype: int64


iris_50
0    25
1    25
2    25
Name: 0, dtype: int64
0    25
1    25
2    25
Name: 0, dtype: int64


gendata_33
0    171
1    164
Name: 0, dtype: int64
0    78
1    87
Name: 0, dtype: int64




In [14]:
print iris_train_data_10.describe()
print iris_test_data_10.describe()

print gendata_train_data_10.describe()
print gendata_test_data_10.describe()

                0           1           2           3
count  135.000000  135.000000  135.000000  135.000000
mean     5.842963    3.062222    3.766667    1.207407
std      0.837867    0.422124    1.776295    0.760953
min      4.300000    2.000000    1.000000    0.100000
25%      5.100000    2.800000    1.600000    0.300000
50%      5.800000    3.000000    4.400000    1.300000
75%      6.400000    3.300000    5.100000    1.800000
max      7.900000    4.400000    6.900000    2.500000
               0          1          2          3
count  15.000000  15.000000  15.000000  15.000000
mean    5.846667   2.980000   3.686667   1.120000
std     0.760514   0.537454   1.710834   0.805517
min     4.800000   2.200000   1.400000   0.100000
25%     5.150000   2.600000   1.550000   0.300000
50%     5.800000   3.100000   4.100000   1.000000
75%     6.400000   3.150000   5.000000   1.750000
max     7.100000   4.100000   5.900000   2.300000
               0           1           2           3           4

The next set of cross-validation split generators deal with folds - so instead of giving you a single set of outputs, they'll return an *iterator* that produces a series of train and test splits.

## Question 2: Cross-validation with k folds (10 points)
Use the `KFold` cross-validation function to generate multiple train-test splits from each dataset
1.   Generate 3, 5, and 10 folds for each dataset (using `random_state=20160121` as a parameter) 
2.   Compute the percentage of the training labels and testing labels that belong to each class for each set of folds. Does the distribution of labels appear to be well-matched between the train and test set?
3.  Compute the mean value for each feature in the train and test set for each fold in the 10-fold split. Which feature has the largest difference between train and test for each of the ten folds? What is the largest difference of the averaged means for the ten folds?

<h1> Question 2 Answers </h1>
<p>1. See bellow </p>
<p>2. The percentagaes seem to appear well matched between train and test for all the different folds</p>
<p>3. For IRIS dataset the largest difference between train and test mean is <b>Fold 6</b> and feature 3 (petal length) with difference of 0.69037
<br>Largest difference in the averaged means for the ten folds is in feature 2, sepal width</p>

In [15]:
print pd.DataFrame(iris.target).size

150


In [16]:
from sklearn.cross_validation import KFold
    
all_datasets = {'iris': {'data': pd.DataFrame(iris.data), 'labels': pd.DataFrame(iris.target)}, 
            'gen': { 'data': gendata_data, 'labels': gendata_targets} }
        
for dataset in all_datasets.keys():
    for nfolds in [3, 5, 10]:
        kfolds = cross_validation.KFold(all_datasets[dataset]['labels'].size, n_folds=nfolds, shuffle=True, random_state=20160121)
        fold = 0
        for train, test in kfolds:
                fold += 1
#                 print ("%s %s" % (train,test))
                if (dataset == 'iris'):
                    train_fold = pd.DataFrame(iris.target[train])
                    test_fold = pd.DataFrame(iris.target[test])
                else:
                    train_fold = pd.DataFrame(gendata_t[train])
                    test_fold = pd.DataFrame(gendata_t[test])         
                    
                print "Dataset: " + dataset + " - nfold: " + str(fold) + "/" + str(nfolds)
                print "Train data"
                print train_fold[0].value_counts(sort=False)
                print "Test data"
                print test_fold[0].value_counts(sort=False)
                print "\n"

                    

Dataset: iris - nfold: 1/3
Train data
0    32
1    34
2    34
Name: 0, dtype: int64
Test data
0    18
1    16
2    16
Name: 0, dtype: int64


Dataset: iris - nfold: 2/3
Train data
0    35
1    35
2    30
Name: 0, dtype: int64
Test data
0    15
1    15
2    20
Name: 0, dtype: int64


Dataset: iris - nfold: 3/3
Train data
0    33
1    31
2    36
Name: 0, dtype: int64
Test data
0    17
1    19
2    14
Name: 0, dtype: int64


Dataset: iris - nfold: 1/5
Train data
0    40
1    39
2    41
Name: 0, dtype: int64
Test data
0    10
1    11
2     9
Name: 0, dtype: int64


Dataset: iris - nfold: 2/5
Train data
0    40
1    39
2    41
Name: 0, dtype: int64
Test data
0    10
1    11
2     9
Name: 0, dtype: int64


Dataset: iris - nfold: 3/5
Train data
0    42
1    42
2    36
Name: 0, dtype: int64
Test data
0     8
1     8
2    14
Name: 0, dtype: int64


Dataset: iris - nfold: 4/5
Train data
0    39
1    42
2    39
Name: 0, dtype: int64
Test data
0    11
1     8
2    11
Name: 0, dtype: int64


Datase

In [17]:
from sklearn.cross_validation import KFold
    
all_datasets = {'iris': {'data': pd.DataFrame(iris.data), 'labels': pd.DataFrame(iris.target)}, 
            'gen': { 'data': gendata_data, 'labels': gendata_targets} }
        
for dataset in all_datasets.keys():
    for nfolds in [10]:
        kfolds = cross_validation.KFold(all_datasets[dataset]['labels'].size, n_folds=nfolds, shuffle=True, random_state=20160121)
        fold = 0
        for train, test in kfolds:
                fold += 1
#                 print ("%s %s" % (train,test))
                if (dataset == 'iris'):
                    train_fold = pd.DataFrame(iris.data[train])
                    test_fold = pd.DataFrame(iris.data[test])
                else:
                    train_fold = pd.DataFrame(gendata_d[train])
                    test_fold = pd.DataFrame(gendata_d[test])         
                    
                print "Dataset: " + dataset + " - nfold: " + str(fold) + "/" + str(nfolds)
                print "Train data"
                print train_fold.describe()
                print "Test data"
                print test_fold.describe()
                print "\n"



Dataset: iris - nfold: 1/10
Train data
                0           1           2           3
count  135.000000  135.000000  135.000000  135.000000
mean     5.842963    3.062222    3.766667    1.207407
std      0.837867    0.422124    1.776295    0.760953
min      4.300000    2.000000    1.000000    0.100000
25%      5.100000    2.800000    1.600000    0.300000
50%      5.800000    3.000000    4.400000    1.300000
75%      6.400000    3.300000    5.100000    1.800000
max      7.900000    4.400000    6.900000    2.500000
Test data
               0          1          2          3
count  15.000000  15.000000  15.000000  15.000000
mean    5.846667   2.980000   3.686667   1.120000
std     0.760514   0.537454   1.710834   0.805517
min     4.800000   2.200000   1.400000   0.100000
25%     5.150000   2.600000   1.550000   0.300000
50%     5.800000   3.100000   4.100000   1.000000
75%     6.400000   3.150000   5.000000   1.750000
max     7.100000   4.100000   5.900000   2.300000


Dataset: iris

Sometimes it's important to ensure you get equal samples of each class, particularly when the classes are imbalanced. Let's generate some data with a label skew and see what happens in cross-validation.


In [18]:
#Let's make some skewed data where the instances with label 1 are very rare:
(skew_gendata_d, skew_gendata_t) = datasets.make_classification(n_samples=500, weights=[0.9], random_state=20160121)
skew_gendata_data = pd.DataFrame(skew_gendata_d)
skew_gendata_targets = pd.DataFrame(skew_gendata_t)

## Question 3: Stratified folds (5 points)
1.  Use the `KFold` cross-validation method and generate 10 folds (using `random_state=20160121`). What is the range of percentages for label 1 across the folds?
2.  Use the `StratifiedKFold` cross-validation method and generate 10 folds (using `random_state=20160121`). What is the range of percentages for label 1 across the folds?

<h1> Question 3 Answer </h1>
<p>1. The range of percentages for label 1 is from 10.22% to 11.11% for the train data and 
from 8% to 16% for the test data</p>
<p>2. The range of percentages for label 1 is 
from 10.69% to 10.86% for the train data 
from 10% to 11.76% for the test data </p>

In [19]:
all_datasets = {'skew': { 'data': skew_gendata_data, 'labels': skew_gendata_targets} }
        
for dataset in all_datasets.keys():
    for nfolds in [10]:
        kfolds = cross_validation.KFold(all_datasets[dataset]['labels'].size, n_folds=nfolds, shuffle=True, random_state=20160121)
        fold = 0
        for train, test in kfolds:
            fold += 1
            train_fold = pd.DataFrame(skew_gendata_t[train])
            test_fold = pd.DataFrame(skew_gendata_t[test])

            print "Dataset: " + dataset + " - nfold: " + str(fold) + "/" + str(nfolds)
            print "Train data"
            print train_fold[0].value_counts(sort=False)
            print "Test data"
            print test_fold[0].value_counts(sort=False)
            print "\n"

Dataset: skew - nfold: 1/10
Train data
0    401
1     49
Name: 0, dtype: int64
Test data
0    45
1     5
Name: 0, dtype: int64


Dataset: skew - nfold: 2/10
Train data
0    403
1     47
Name: 0, dtype: int64
Test data
0    43
1     7
Name: 0, dtype: int64


Dataset: skew - nfold: 3/10
Train data
0    403
1     47
Name: 0, dtype: int64
Test data
0    43
1     7
Name: 0, dtype: int64


Dataset: skew - nfold: 4/10
Train data
0    400
1     50
Name: 0, dtype: int64
Test data
0    46
1     4
Name: 0, dtype: int64


Dataset: skew - nfold: 5/10
Train data
0    400
1     50
Name: 0, dtype: int64
Test data
0    46
1     4
Name: 0, dtype: int64


Dataset: skew - nfold: 6/10
Train data
0    404
1     46
Name: 0, dtype: int64
Test data
0    42
1     8
Name: 0, dtype: int64


Dataset: skew - nfold: 7/10
Train data
0    403
1     47
Name: 0, dtype: int64
Test data
0    43
1     7
Name: 0, dtype: int64


Dataset: skew - nfold: 8/10
Train data
0    400
1     50
Name: 0, dtype: int64
Test data
0    46


In [20]:
# print skew_gendata_targets.shape
print np.reshape(skew_gendata_targets.values,[500,]).shape

(500,)


In [21]:
# Use the StratifiedKFold cross-validation method and generate 10 folds (using random_state=20160121). 
# What is the range of percentages for label 1 across the folds?
from sklearn.cross_validation import StratifiedKFold

kfolds = cross_validation.StratifiedKFold(np.reshape(skew_gendata_targets.values,[500,]), n_folds=10, shuffle=True, random_state=20160121)
fold = 0
for train, test in kfolds:
    fold += 1
    train_fold = pd.DataFrame(skew_gendata_t[train])
    test_fold = pd.DataFrame(skew_gendata_t[test])

    print "Dataset: " + dataset + " - nfold: " + str(fold) + "/" + str(nfolds)
    print "Train data"
    print train_fold[0].value_counts(sort=False)
    print "Test data"
    print test_fold[0].value_counts(sort=False)
    print "\n"


Dataset: skew - nfold: 1/10
Train data
0    401
1     48
Name: 0, dtype: int64
Test data
0    45
1     6
Name: 0, dtype: int64


Dataset: skew - nfold: 2/10
Train data
0    401
1     48
Name: 0, dtype: int64
Test data
0    45
1     6
Name: 0, dtype: int64


Dataset: skew - nfold: 3/10
Train data
0    401
1     48
Name: 0, dtype: int64
Test data
0    45
1     6
Name: 0, dtype: int64


Dataset: skew - nfold: 4/10
Train data
0    401
1     48
Name: 0, dtype: int64
Test data
0    45
1     6
Name: 0, dtype: int64


Dataset: skew - nfold: 5/10
Train data
0    401
1     49
Name: 0, dtype: int64
Test data
0    45
1     5
Name: 0, dtype: int64


Dataset: skew - nfold: 6/10
Train data
0    401
1     49
Name: 0, dtype: int64
Test data
0    45
1     5
Name: 0, dtype: int64


Dataset: skew - nfold: 7/10
Train data
0    402
1     49
Name: 0, dtype: int64
Test data
0    44
1     5
Name: 0, dtype: int64


Dataset: skew - nfold: 8/10
Train data
0    402
1     49
Name: 0, dtype: int64
Test data
0    44


One of the techniques we discussed was the bootstrap: a method to get a training set the size of our dataset while still getting useful training data. You can use the [resample](http://scikit-learn.org/stable/modules/generated/sklearn.utils.resample.html) method from sklearn's util module to implement the bootstrap. Resample will generate a training set for you and, by default, make it the same size as the dataset. The only remaining challenge is figuring out what to put in the test set. You can use the `index` attribute of a pandas DataFrame to help you there...

## Question 4: The Bootstrap (10 points)
1.  Generate a bootstrap based train-test split for the three datasets (gendata, skew_gendata, and iris)  using the `resample` method. Remember to set `random_state=20160121`
2.  Compute the label proportions in the train and test sets. How do they compare?
3.  Compute the descriptive statistics for each of the attributes. Which attribute has the largest difference in mean across the train and test sets?

<h1> Question 4 Answers </h1>
<p>1. See code below </p>
<p>2. Gendata label proportions are not great, but they are close enough to be considered okay for our experiments.<br>
See cell bellow for formatting</p>
<p>3. 15th attribute has the largest difference in mean across the train and test sets </p>


In [22]:
from sklearn.utils import resample

boot_gendata_train = resample(gendata_targets, random_state=20160121)
boot_gendata_test = gendata_targets.loc[~gendata_targets.index.isin(list(boot_gendata_train.index.values))]

# print boot_gendata_train
# print boot_gendata_test

print "Train data"
print boot_gendata_train[0].value_counts(sort=False)
print "Test data"
print boot_gendata_test[0].value_counts(sort=False)
print "\n"

boot_gendata_train = resample(gendata_data, random_state=20160121)
boot_gendata_test = gendata_data.loc[~gendata_data.index.isin(list(boot_gendata_train.index.values))]

print boot_gendata_train.describe()
print boot_gendata_test.describe()

Train data
0    259
1    241
Name: 0, dtype: int64
Test data
0    80
1    95
Name: 0, dtype: int64


               0           1           2           3           4           5   \
count  500.000000  500.000000  500.000000  500.000000  500.000000  500.000000   
mean    -0.003139   -0.013427    0.021906    0.035423    0.012670    0.096852   
std      0.991457    1.020868    1.018835    0.981776    1.041781    1.003704   
min     -2.421082   -2.433603   -2.538242   -2.518441   -3.223148   -2.536563   
25%     -0.720644   -0.757001   -0.615010   -0.571672   -0.740107   -0.535868   
50%     -0.013046   -0.070751   -0.000330    0.008836    0.047413    0.171539   
75%      0.661775    0.620804    0.679261    0.723654    0.644627    0.862462   
max      3.594499    3.039572    3.003953    2.619359    2.454954    3.432993   

               6           7           8           9           10          11  \
count  500.000000  500.000000  500.000000  500.000000  500.000000  500.000000   
mean   

In [23]:
from sklearn.utils import resample

boot_skew_gendata_train = resample(skew_gendata_targets, random_state=20160121)
boot_skew_gendata_test = skew_gendata_targets.loc[~skew_gendata_targets.index.isin(list(boot_skew_gendata_train.index.values))]

# print boot_skew_gendata_train.shape
# print boot_skew_gendata_test.shape

print "Train data Skew"
print boot_skew_gendata_train[0].value_counts(sort=False)
print "Test data Skew"
print boot_skew_gendata_test[0].value_counts(sort=False)
print "\n"

boot_skew_gendata_train = resample(skew_gendata_data, random_state=20160121)
boot_skew_gendata_test = skew_gendata_data.loc[~skew_gendata_data.index.isin(list(boot_skew_gendata_train.index.values))]

print boot_skew_gendata_train.describe()
print boot_skew_gendata_test.describe()

Train data Skew
0    458
1     42
Name: 0, dtype: int64
Test data Skew
0    149
1     26
Name: 0, dtype: int64


               0           1           2           3           4           5   \
count  500.000000  500.000000  500.000000  500.000000  500.000000  500.000000   
mean    -0.003139   -0.536252    0.021906    0.035423    0.012670    0.096852   
std      0.991457    0.767778    1.018835    0.981776    1.041781    1.003704   
min     -2.421082   -2.433603   -2.538242   -2.518441   -3.223148   -2.536563   
25%     -0.720644   -1.192733   -0.615010   -0.571672   -0.740107   -0.535868   
50%     -0.013046   -0.456621   -0.000330    0.008836    0.047413    0.171539   
75%      0.661775    0.031560    0.679261    0.723654    0.644627    0.862462   
max      3.594499    1.763571    3.003953    2.619359    2.454954    3.432993   

               6           7           8           9           10          11  \
count  500.000000  500.000000  500.000000  500.000000  500.000000  500.00000

In [24]:
from sklearn.utils import resample

iris_data = pd.DataFrame(iris.target)
boot_iris_train = resample(iris_data, random_state=20160121)

boot_iris_test = iris_data.loc[~iris_data.index.isin(list(boot_iris_train.index.values))]

print "Train data Iris "
print boot_iris_train[0].value_counts(sort=False)
print "Test data Iris"
print boot_iris_test[0].value_counts(sort=False)
print "\n"

iris_data = pd.DataFrame(iris.data)
boot_iris_train = resample(iris_data, random_state=20160121)

boot_iris_test = iris_data.loc[~iris_data.index.isin(list(boot_iris_train.index.values))]

print "Train data Iris"
print boot_iris_train.describe()
print "\nTest data Iris"
print boot_iris_test.describe()
print "\n"

Train data Iris 
0    54
1    50
2    46
Name: 0, dtype: int64
Test data Iris
0    20
1    16
2    18
Name: 0, dtype: int64


Train data Iris
                0           1           2           3
count  150.000000  150.000000  150.000000  150.000000
mean     5.815333    3.018000    3.649333    1.128667
std      0.782590    0.398162    1.731498    0.748810
min      4.400000    2.000000    1.000000    0.100000
25%      5.100000    2.800000    1.500000    0.200000
50%      5.800000    3.000000    4.350000    1.300000
75%      6.375000    3.300000    5.000000    1.800000
max      7.700000    4.400000    6.900000    2.500000

Test data Iris
               0          1          2          3
count  54.000000  54.000000  54.000000  54.000000
mean    5.783333   3.114815   3.618519   1.157407
std     0.855559   0.459864   1.845042   0.802261
min     4.300000   2.200000   1.100000   0.100000
25%     5.100000   2.800000   1.500000   0.225000
50%     5.600000   3.000000   4.050000   1.300000
75%   

## Evaluating Models
Now that we've looked at various means of splitting our data, we can explore the performance metrics used to evaluate our results. You can find a full list of these metrics documented in the [metrics documentation](http://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics). Since it's not very interesting to look at these metrics in isolation, we can generate some classification output for our data and see how the different methods perform.

One of the other topics we discussed was comparing schemes or methods to decide if the differences between them were statistically meaningful. We evaluate the statistical significance of these results using t-tests. The t-test isn't in scikit-learn, but in a related module, `scipy`. You can read about all of the statistical tests in the [scipy stats documentation](http://docs.scipy.org/doc/scipy/reference/stats.html) 


In [26]:
# Train the dummy classifier on the Iris data:
dummy_classifier_iris = dummy.DummyClassifier();
dummy_classifier_iris.fit(iris_train_data, iris_train_labels);
dummy_iris_predictions = dummy_classifier_iris.predict(iris_test_data);
dummy_iris_predictions_proba = dummy_classifier_iris.predict_proba(iris_test_data);

#Train the decision tree classifier on the Iris data
dtree_iris = tree.DecisionTreeClassifier();
dtree_iris.fit(iris_train_data, iris_train_labels);
dtree_iris_predictions = dtree_iris.predict(iris_test_data);
dtree_iris_predictions_proba = dtree_iris.predict_proba(iris_test_data);

#The classification report is a handy tool to see many metrics from one command
print "Dummy classifier report"
print metrics.classification_report(iris_test_labels, dummy_iris_predictions)

print "Iris classifier report"
print metrics.classification_report(iris_test_labels, dtree_iris_predictions)


print dummy_iris_predictions
print dtree_iris_predictions
#We can compare the two predictions using the t-test (from scipy)
stats.ttest_rel(dummy_iris_predictions, dtree_iris_predictions)

Dummy classifier report
             precision    recall  f1-score   support

          0       0.31      0.31      0.31        13
          1       0.30      0.23      0.26        13
          2       0.20      0.25      0.22        12

avg / total       0.27      0.26      0.26        38

Iris classifier report
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        13
          1       0.80      0.92      0.86        13
          2       0.90      0.75      0.82        12

avg / total       0.90      0.89      0.89        38

[0 2 0 0 0 0 1 2 1 0 1 1 2 2 2 0 2 2 1 1 2 1 0 0 0 1 2 2 2 2 1 0 2 2 1 0 2
 0]
[0 1 1 0 2 1 1 1 0 0 2 2 1 2 0 1 1 0 1 1 1 1 2 0 1 2 0 0 0 1 2 1 0 0 2 2 0
 2]


Ttest_relResult(statistic=0.61507975774240686, pvalue=0.54226679409042178)

## Question 5: Evaluation Metrics: Discrete Predictions (10 points)
1.   Compute the accuracy for each classifier
2.   Make a simple train-test split of gendata (using `randomstate=20160121`) and perform the same analysis: classification report and t-test
3.   One of the issues in realistic evaluation settings is that the test outputs are not always generated in the same testing regime. Perform 5-fold cross-validation using the DummyClassifier and 10-fold cross-validation using the DecisionTree classifier. Read the documentation to determine the correct t-test function to use to compare these two sets of results, and perform the t-test across folds

<h1> Question 5 Answers </h1>
<p>1. Dummy Classifier accuracy: 0.421052631579 <br>
Decision Tree Classifier accuracy: 0.894736842105</p>
<p>2. Ttest_relResult(statistic=-0.74098388608089094, pvalue=0.45976285530296646)

see cell below for better formattting </p>
<p>3. Ttest_indResult(statistic=14.289053827191161, pvalue=2.509241547322248e-09)</p>

In [61]:
# part 1
from sklearn.metrics import accuracy_score

print "Dummy Classifier accuracy: " + str(accuracy_score(iris_test_labels, dummy_iris_predictions))
print "Decision Tree Classifier accuracy: " + str(accuracy_score(iris_test_labels, dtree_iris_predictions))


Dummy Classifier accuracy: 0.289473684211
Decision Tree Classifier accuracy: 0.894736842105


In [62]:
# part 2 
[gendata_train_data_33, gendata_test_data_33,  gendata_train_labels_33, gendata_test_labels_33] = cross_validation.train_test_split(gendata_d, gendata_t, test_size=0.33, random_state=20160121)

dummy_classifier_gendata = dummy.DummyClassifier();
dummy_classifier_gendata.fit(gendata_train_data_33, gendata_train_labels_33)
dummy_gendata_predictions = dummy_classifier_gendata.predict(gendata_test_data_33)
dummy_gendata_predictions_proba = dummy_classifier_gendata.predict_proba(gendata_test_data_33)

dtree_gendata = tree.DecisionTreeClassifier();
dtree_gendata.fit(gendata_train_data_33, gendata_train_labels_33)
dtree_gendata_predictions = dtree_gendata.predict(gendata_test_data_33)
dtree_gendata_predictions_prob = dtree_gendata.predict_proba(gendata_test_data_33)

#The classification report is a handy tool to see many metrics from one command
print "Dummy classifier report"
print metrics.classification_report(gendata_test_labels_33, dummy_gendata_predictions)

print "Iris classifier report"
print metrics.classification_report(gendata_test_labels_33, dtree_gendata_predictions)

#We can compare the two predictions using the t-test (from scipy)
print stats.ttest_rel(dummy_gendata_predictions, dtree_gendata_predictions)


Dummy classifier report
             precision    recall  f1-score   support

          0       0.48      0.54      0.51        78
          1       0.53      0.47      0.50        87

avg / total       0.51      0.50      0.50       165

Iris classifier report
             precision    recall  f1-score   support

          0       0.81      0.82      0.82        78
          1       0.84      0.83      0.83        87

avg / total       0.82      0.82      0.82       165

Ttest_relResult(statistic=-0.9431404248511186, pvalue=0.34699638241967168)


In [63]:
# part 3
from sklearn.metrics import accuracy_score
kfolds = cross_validation.KFold(500, n_folds=5, shuffle=True, random_state=20160121)
fold = 0

dummy_accuracy = []
for train, test in kfolds:
    fold += 1
    train_fold = gendata_d[train]
    train_labels = gendata_t[train]
    
    test_fold = gendata_d[test]
    test_labels = gendata_t[test]
    
    dummy_classifier_gendata = dummy.DummyClassifier();
    dummy_classifier_gendata.fit(train_fold, train_labels)
    dummy_gendata_predictions = dummy_classifier_gendata.predict(test_fold)
    dummy_gendata_predictions_proba = dummy_classifier_gendata.predict_proba(test_fold)
    
    score = 1 - accuracy_score(test_labels, dummy_gendata_predictions)
#     score = accuracy_score(test_labels, dummy_gendata_predictions)
    dummy_accuracy.append(score)

dtree_accuracy = []
kfolds = cross_validation.KFold(500, n_folds=10, shuffle=True, random_state=20160121)
for train, test in kfolds:
    fold += 1
    train_fold = gendata_d[train]
    train_labels = gendata_t[train]
    
    test_fold = gendata_d[test]
    test_labels = gendata_t[test]
    
    dtree_gendata = tree.DecisionTreeClassifier();
    dtree_gendata.fit(train_fold, train_labels)
    dtree_gendata_predictions = dtree_gendata.predict(test_fold)
    dtree_gendata_predictions_prob = dtree_gendata.predict_proba(test_fold)
    
    score = 1 - accuracy_score(test_labels, dtree_gendata_predictions)
#     score = accuracy_score(test_labels, dtree_gendata_predictions)
    dtree_accuracy.append(score)
    
#We can compare the two predictions using the t-test (from scipy)
print dummy_accuracy
print dtree_accuracy
stats.ttest_ind(dummy_accuracy, dtree_accuracy)

[0.48999999999999999, 0.48999999999999999, 0.40000000000000002, 0.53000000000000003, 0.60999999999999999]
[0.19999999999999996, 0.07999999999999996, 0.099999999999999978, 0.07999999999999996, 0.099999999999999978, 0.16000000000000003, 0.020000000000000018, 0.060000000000000053, 0.14000000000000001, 0.040000000000000036]


Ttest_indResult(statistic=11.871860348739082, pvalue=2.3767018632247007e-08)